In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
import os
import shutil
import pickle  # For saving FAISS index

In [5]:
import faiss

## Load and Split Documents

In [5]:
Data_Path = r"D:\WorkPlace\Python\Training\May2025\Practice\PracLLM"
FAISS_Path = r"D:\WorkPlace\Python\Training\May2025\Practice\PracLLM\FAISS_DB\UFAREX"

In [7]:
loader = PyPDFDirectoryLoader(Data_Path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " "]
)
chunks = text_splitter.split_documents(documents)

In [9]:
print(type(chunks), len(chunks))

<class 'list'> 93


In [11]:
print((chunks[0]))

page_content='IEEE TRANSACTIONS ON SYSTEMS, MAN, AND CYBERNETICS: SYSTEMS, VOL. 54, NO. 12, DECEMBER 2024 7419
UFAREX: A Universal Fully Autonomous Robust
Expansionist Fuzzy System for Optimal Online
Learning From Nonstationary Data Streams
Cyrus Hasanvand , Hamid Hasanvand , and Hamidreza Momeni , Senior Member, IEEE
Abstract—Online intelligent knowledge extraction from real-
world nonstationary data streams presents a multiobjective
optimization challenge. Here, we characterize the learning pro-
cess on a trajectory of global optimality to simultaneously satisfy
six high-proﬁle objectives: 1) optimum generalization for the
best bias-variance tradeoff; 2) compactness of knowledgebase;
3) memory retention and stability-plasticity balance; 4) univer-
sality and full autonomy; 5) robustness against outliers, noise,
and model uncertainty; and 6) active concept drift detection
and adaptation. We propose a ﬂexible Takagi–Sugeno (TS) fuzzy
system, named UFAREX, that self-constructs and self-

## Embed and Build FAISS Vector Store

In [16]:
#embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# Create FAISS vector store
vectorstore = FAISS.from_documents(chunks, embedding=embedding_model)

## Save the FAISS DB (Persistence)

Unlike Chroma (which has persist_directory), FAISS requires manual saving using faiss + pickle.

In [31]:
if os.path.exists(FAISS_Path):
    shutil.rmtree(FAISS_Path) # Delete the folder and its contents
os.makedirs(FAISS_Path, exist_ok=True) # Re-create the folder

# Save index
index_path = os.path.join(FAISS_Path, "index.faiss")
faiss.write_index(vectorstore.index, index_path)

# Save documents and embeddings separately
store_path = os.path.join(FAISS_Path, "store.pkl")
with open(store_path, "wb") as f:
    pickle.dump(vectorstore, f)

print(f"Saved {len(chunks)} chunks to {FAISS_Path}")

Saved 93 chunks to D:\WorkPlace\Python\Training\May2025\Practice\PracLLM\FAISS_DB\UFAREX


## Load the FAISS DB Later

When you want to reload your saved FAISS database:

In [7]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import faiss
import pickle

FAISS_Path = r"D:\WorkPlace\Python\Training\May2025\Practice\PracLLM\FAISS_DB\UFAREX"

index_path = os.path.join(FAISS_Path, "index.faiss")
store_path = os.path.join(FAISS_Path, "store.pkl")

with open(store_path, "rb") as f:
    vectorstore = pickle.load(f)

## Add New Documents to Existing FAISS VectorStore
### Load or prepare new documents
I already have __**vectorstore**__ from previous cell.

Use any method to get new chunks. For example:

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

New_Data_Path = r"D:\NewDocuments"

loader = PyPDFDirectoryLoader(New_Data_Path)
new_docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

new_chunks = text_splitter.split_documents(new_docs)

### Add new chunks to the existing vectorstore
This step embeds the new chunks using the original embedding model (automatically stored in the vectorstore) and adds them to the FAISS index.

In [ ]:
vectorstore.add_documents(new_chunks)

### Save the updated vectorstore again

In [ ]:
import faiss
import pickle
import os

# Save updated FAISS index
index_path = os.path.join(FAISS_Path, "index.faiss")
faiss.write_index(vectorstore.index, index_path)

# Save updated full store
store_path = os.path.join(FAISS_Path, "store.pkl")
with open(store_path, "wb") as f:
    pickle.dump(vectorstore, f)

## Load for search

In [60]:
Query="definition of nonstationary data streams (NDS)"
Results = vectorstore.similarity_search(Query, k=5)

In [62]:
print(type(Results))
print(type(Results[0].page_content))
#print((results[0].page_content))
print((Results[0].metadata['source']))

print((Results[0]))

<class 'list'>
<class 'str'>
D:\WorkPlace\Python\Training\May2025\Practice\PracLLM\Cyrus Hasanvand - 2024_ UFAREX_ IEEE Trans on SMCA.pdf
page_content='agement with zero buffer latency for regression applications. No
heuristic forgetting, pruning, splitting, merging, and weighting
mechanisms are exercised to prevent human intervention and
render universality. UFAREX was comparatively tested on four
real-world benchmarks. It stands out as an autonomous system
geared for adaptive modeling, time-series forecasting, anomaly
monitoring, and robust fault detection and diagnosis.
Index Terms—Active concept drift detection and adaptation,
adaptive prediction intervals (APIs), catastrophic forgetting (CF),
noise and outlier, nonstationary data streams (NDSs), uncertainty.
I. I NTRODUCTION
O
NLINE knowledge discovery from real-world non-
stationary data streams (NDSs) presents profound
challenges to the design of modern computational intelligence.
Manuscript received 4 June 2024; accepted 20 Aug

## LLM

In [75]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
#from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.schema import SystemMessage, HumanMessage, AIMessage, StrOutputParser
from langchain_ollama import ChatOllama
#from langchain_huggingface import HuggingFacePipeline 

In [77]:
LLM = ChatOllama(model="llama3.1", Temperature=0.7,do_sample=True)

In [81]:
prompt=ChatPromptTemplate.from_messages([
    ('system','You are an AI assistant, which refines the orders of similarity search from FAISS database and provides no more than 220 tokens when writing a response to a question'),
    ('user','{text}')
])

In [83]:
ChainLLama31=prompt|LLM|StrOutputParser()

In [99]:
Question = f'''
You are given the results of a FAISS similarity search. 
The user asked the following query:
Query: "{Query}"

Here are the retrieved results:
Result[0]: {Results[0].page_content}
Result[1]: {Results[1].page_content}
Result[2]: {Results[2].page_content}
Result[3]: {Results[3].page_content}
Result[4]: {Results[4].page_content}

Please re-order these results in order of **relevance to the user's query**,
starting with the most relevant. 
In your answer, you can refer to them by their original indices, e.g., Result[0],
Result[1], etc., and optionally explain why each result is in that position.
'''

In [101]:
Response_message=ChainLLama31.invoke({'text':Question})

In [103]:
print(Response_message)

After analyzing the results, I've reordered them based on relevance to the user's query "definition of nonstationary data streams (NDS)". Here are the results:

1. **Result[1]**: The core theme of NDS is concept drift. It results in conflict between the old and new trends... This result directly answers the user's question by defining a key aspect of NDS.
2. **Result[0]**: Online knowledge discovery from real-world non-stationary data streams (NDSs) presents profound challenges to the design of modern computational intelligence... While this result doesn't explicitly define NDS, it mentions it and discusses its challenges, making it highly relevant.
3. **Result[4]**: ...the sole dominant rule undertakes the prediction task... This result is less directly related to defining NDS, but it does discuss a concept drift within the context of NDS, which makes it somewhat relevant.
4. **Result[2]**: Thus, the sole dominant rule undertakes the prediction task... This result is similar to Result